# Phase Estimation

The **"Phase Estimation"** quantum kata is a series of exercises designed
to teach you the basics of using phase estimation algorithms.

It covers the following topics:
* quantum phase estimation,
* iterative phase estimation,
* preparing necessary inputs to phase estimation routines and applying them.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Quantum Phase Estimation (QPE)

### Task 1.1. Inputs to QPE: eigenstates of Z/S/T gates.

**Inputs:** 

  1. A qubit in the $|0\rangle$ state.

  2. An integer `state` indicating which eigenstate to prepare.

**Goal:** 

Prepare one of the eigenstates of Z gate (which are the same as eigenstates of S or T gates): 
eigenstate $|0\rangle$ if `state = 0`, or eigenstate $|1\rangle$ if `state = 1`.

In [ ]:
%kata T11_Eigenstates_ZST 

operation Eigenstates_ZST (q : Qubit, state : Int) : Unit is Adj {
    // ...
    if state == 1 {
        X(q);
    }
}

Success!

### Task 1.2. Inputs to QPE: powers of Z/S/T gates.

**Inputs:** 

  1. A single-qubit unitary U.

  2. A positive integer `power`.

**Output:** 

A single-qubit unitary equal to U raised to the given power.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Remember that you can define auxiliary operations. To do that, you'll need to create an extra code cell for each new operation and execute it before returning to this cell. 
</details>

In [ ]:
operation UnitaryPowerHelper(U: (Qubit => Unit is Adj+Ctl), power: Int, q: Qubit): Unit is Adj + Ctl {
    for i in 1..power {
        U(q);
    }
}

UnitaryPowerHelper

In [ ]:
%kata T12_UnitaryPower 

function UnitaryPower (U : (Qubit => Unit is Adj + Ctl), power : Int) : (Qubit => Unit is Adj + Ctl) {
    // ...
    return UnitaryPowerHelper(U, power, _);
}

Success!

### Task 1.3. Validate inputs to QPE.

<span style="color:red"><b>This task is temporarily not available in Notebook format; please use Q# project version of the PhaseEstimation kata to complete it.</b></span>

**Inputs:**

  1. A single-qubit unitary U.

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

**Goal:** 

Assert that the given state is an eigenstate of the given unitary, 
i.e., do nothing if it is, and throw an exception if it is not.

### Task 1.4. QPE for single-qubit unitaries.

**Inputs:** 

  1. A single-qubit unitary U.

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

  3. An integer `n`.

**Output:**

The phase of the eigenvalue that corresponds to the eigenstate $|\psi\rangle$, with `n` bits of precision.
The phase should be between 0.0 and 1.0.

In [ ]:
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

In [ ]:
%config dump.basisStateLabelingConvention = "Bitstring"
%config dump.truncationThreshold = 0.01
%config dump.truncateSmallAmplitudes = true
%config  

Configuration key,Value
dump.basisStateLabelingConvention,"""Bitstring"""
dump.truncationThreshold,0.01
dump.truncateSmallAmplitudes,true


In [ ]:
%kata T14_QPE 

operation QPE (U : (Qubit => Unit is Adj + Ctl), P : (Qubit => Unit is Adj), n : Int) : Double {
    // ...
    use qAux = Qubit[n];
    use qPsi = Qubit();
    //within {
    P(qPsi);
    //}apply{
    ApplyToEach(H, qAux);
    
    for i in 0..n-1 {
        Controlled (UnitaryPower(U, 2^i))([qAux[i]], qPsi);
    }
    
    Adjoint QFTLE(LittleEndian(qAux)); // WHY ???!!
    //}
    //DumpMachine();
    mutable phase = 0.0;
    for i in 0..n-1 {
        let m = M(qAux[n-i-1]) == One ? 1 | 0;
        set phase = phase + IntAsDouble(m)*0.5^IntAsDouble(i+1);
    }
    Reset(qPsi);
    
    return phase;
}

Success!

### Task 1.5. Test your QPE implementation.

**Goal:**
Use your QPE implementation from task 1.4 to run quantum phase estimation 
on several simple unitaries and their eigenstates.
This task is not covered by a test and allows you to experiment with running the algorithm.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_QPE` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_QPE`).

In [ ]:
operation Run_QPE () : Unit {
    // ...
}

In [ ]:
%simulate Run_QPE

## Part II. Iterative Phase Estimation

Unlike quantum phase estimation, which is a single algorithm, 
iterative phase estimation is a whole class of algorithms based on the same idea:
treating phase estimation as a classical algorithm which learns the phase via a sequence of measurements
(the measurement performed on each iteration can depend on the outcomes of previous iterations).

A typical circuit for one iteration has the following structure:

![Iterative Phase Estimation Circuit Diagram](./img/IPE_Circuit.PNG)

($\psi$ is the procedure to prepare the eigenstate $|\psi\rangle$, R is a rotation gate, and M is a power of the unitary U;
both depend on the current information about the phase).

The result of the measurement performed on the top qubit defines the next iteration.

### Task 2.1. Single-bit phase estimation.

**Inputs:** 

  1. A single-qubit unitary U that is guaranteed to have an eigenvalue $+1$ or $-1$ 
(with eigenphases $0.0$ or $0.5$, respectively).

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

**Output:** 

The eigenvalue which corresponds to the eigenstate $|\psi\rangle$ ($+1$ or $-1$).

You are allowed to allocate exactly two qubits and call `Controlled U` exactly once.

> It is possible to use the QPE implementation from task 1.4 to solve this task,
  but we suggest you implement the circuit by hand for the sake of learning.

In [ ]:
%kata T21_SingleBitPE 

operation SingleBitPE (U : (Qubit => Unit is Adj + Ctl), P : (Qubit => Unit is Adj)) : Int {
    // ...
    use (qPha, qPsi) = (Qubit(), Qubit());
    P(qPsi);
    H(qPha);
    Z(qPha);
    Controlled U([qPha], qPsi);
    H(qPha);
    let m = M(qPha) == One ? 1 | -1;
    Reset(qPsi);
    //DumpMachine();
    return m;
}

Success!

### Task 2.2. Two bit phase estimation.

**Inputs:**

  1. A single-qubit unitary U that is guaranteed to have an eigenvalue $+1$, $i$, $-1$ or $-i$
(with eigenphases $0.0$, $0.25$, $0.5$ or $0.75$, respectively).

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

**Output:**

The eigenphase which corresponds to the eigenstate $|\psi\rangle$ ($0.0$, $0.25$, $0.5$ or $0.75$).
The returned value has to be accurate within the absolute error of 0.001.

You are allowed to allocate exactly two qubits and call `Controlled U` multiple times.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Start by applying the same circuit as in task 2.1.  
  What are the possible outcomes for each eigenvalue?  
  What eigenvalues you can and can not distinguish using this circuit?
</details>

<br/>
<details>
  <summary><b>Need another hint? Click here</b></summary>
  What eigenvalues you can and can not distinguish using this circuit?
  What circuit you can apply to distinguish them?
</details>

In [ ]:
%kata T22_TwoBitPE 

operation TwoBitPE (U : (Qubit => Unit is Adj + Ctl), P : (Qubit => Unit is Adj)) : Double {
    // ...
    use (qPha, qPsi) = (Qubit(), Qubit());
    P(qPsi);
    H(qPha);
    Controlled U([qPha], qPsi);
    Controlled U([qPha], qPsi);
    H(qPha);
    let mPha = M(qPha);
    Reset(qPsi);
    Reset(qPha);
    
    if (mPha == One) { // Imaginary components 
        P(qPsi);
        H(qPha);
        S(qPha);
        Controlled U([qPha], qPsi);
        H(qPha);
        let output = M(qPha) == Zero? 0.75 | 0.25;
        Reset(qPsi);
        return output;
        
    } else { // mPha = Zero
        P(qPsi);
        H(qPha);
        Controlled U([qPha], qPsi);
        H(qPha);
        let output = M(qPha) == Zero ? 0.0 | 0.5 ;
        Reset(qPsi);
        return output;
    }
    
    // return -1.0;
}

Success!

To be continued...